导入库

In [1]:
import os

import numpy as np

import utils

公共变量

In [2]:
words_path = r'./feat/words'
pts = ['sub-%02d'%i for i in range(1,11)]

读取数据

In [3]:
eeg = dict()
audio = dict()
word = dict()
for pt in pts:
    folder_path = os.path.join(words_path,f'{pt}')
    word[pt] = []
    eeg[pt] = []
    audio[pt] = []
    for filename in os.listdir(folder_path):
        word_info = np.load(os.path.join(folder_path,filename),allow_pickle=True)
        word[pt].append(word_info.item()['label'])
        eeg[pt].append(word_info.item()['eeg'])
        audio[pt].append(word_info.item()['audio'])


In [4]:
for pt in pts:
    print(f'{pt}:',word[pt][0],len(eeg[pt]),eeg[pt][0].shape,audio[pt][0].shape)

sub-01: helft 100 (856, 127) (13376,)
sub-02: ook 100 (512, 127) (8000,)
sub-03: buurt 100 (726, 127) (11346,)
sub-04: vogeltje 100 (1044, 115) (16320,)
sub-05: verlost 100 (901, 60) (14080,)
sub-06: hoe 100 (474, 127) (7405,)
sub-07: zanddak 100 (1017, 127) (15899,)
sub-08: de 100 (358, 54) (5599,)
sub-09: tak 100 (696, 117) (10880,)
sub-10: vijf 95 (367, 122) (5732,)


数据预处理

In [5]:
window_length = 0.025
frameshift = 0.005
eeg_sample_rate = 1024
audio_sameple_rate = 16000

提取高频eeg信号和音频信号的梅尔频谱

In [6]:
for pt in pts:
    for i in range(len(eeg[pt])):
        eeg[pt][i] = utils.extractHG(eeg[pt][i],eeg_sample_rate,windowLength=window_length,frameshift=frameshift)
        audio[pt][i] = utils.extractMelSpecs(audio[pt][i],audio_sameple_rate,windowLength=window_length,frameshift=frameshift)
        if audio[pt][i].shape[0]!=eeg[pt][i].shape[0]:
            minlen = min(audio[pt][i].shape[0],eeg[pt][i].shape[0])
            audio[pt][i] = audio[pt][i][:minlen,:]
            eeg[pt][i] = eeg[pt][i][:minlen,:]

In [7]:
for pt in pts:
    print(f'{pt}:',word[pt][0],len(eeg[pt]),eeg[pt][0].shape,audio[pt][0].shape)

sub-01: helft 100 (162, 127) (162, 23)
sub-02: ook 100 (95, 127) (95, 23)
sub-03: buurt 100 (136, 127) (136, 23)
sub-04: vogeltje 100 (198, 115) (198, 23)
sub-05: verlost 100 (170, 60) (170, 23)
sub-06: hoe 100 (87, 127) (87, 23)
sub-07: zanddak 100 (193, 127) (193, 23)
sub-08: de 100 (64, 54) (64, 23)
sub-09: tak 100 (130, 117) (130, 23)
sub-10: vijf 95 (66, 122) (66, 23)


拼接方便后续处理

In [97]:
pt = pts[5]
total_data = np.array(eeg[pt][0])
total_label = np.array(audio[pt][0])
for i in range(len(eeg[pt])):
    if i == 0:
        continue
    total_data = np.concatenate((total_data,eeg[pt][i]),axis=0)
    total_label = np.concatenate((total_label,audio[pt][i]),axis=0)
print(total_data.shape)

(15308, 127)


z均值处理

In [98]:
# 随机打乱数据索引
indices = np.random.permutation(total_data.shape[0])

# 计算划分索引
split_index = int(total_data.shape[0] * 0.9)

# 划分数据集
train_indices = indices[:split_index]
test_indices = indices[split_index:]

# 获取训练集和测试集数据
train_data = total_data[train_indices]
train_label = total_label[train_indices]
test_data = total_data[test_indices]
test_label = total_label[test_indices]

train_data_mean = np.mean(train_data)
train_data_std = np.std(train_data)

train_label_mean = np.mean(train_label)
train_label_std = np.std(train_label)


train_data = (train_data-train_data_mean)/train_data_std
test_data = (test_data-train_data_mean)/train_data_std

# train_label = (train_label-train_label_mean)/train_label_std
# test_label = (test_label- train_label_std)/train_label_std

print(train_data.shape)
# print(train_data[0])
# print(train_data[1])
print(train_label[0])
print(train_label[1])

(13777, 127)
[10.53307705 11.41864517 12.2387321  12.36208222 11.59930787  9.83395343
 10.03693948 10.57370836  9.92624195  8.90767481  7.49765081  6.11109069
  5.80136929  6.32955009  6.65738444  7.41547172  7.0842294   5.53498275
  6.57884064  5.8545331   5.59856505  5.09560521  2.57025279]
[ 9.77679059 10.86393702 11.69747238 11.98998318 12.07730874 10.92936929
 11.73153468 10.9757972  10.92168166 10.55018863 10.09834642  9.85315717
 10.64005122  9.93200347  8.93915556  8.13008183  7.36004233  6.39690863
  7.23925989  6.84910403  6.5176429   5.59149135  3.35894218]


创建dataloader

In [99]:
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn

from torch.utils.data import TensorDataset, DataLoader

import imp
import transformer
imp.reload(transformer)

<module 'transformer' from 'd:\\workspace\\research\\SingleWordProductionDutch\\transformer.py'>

In [100]:
batch_size = 256
epochs = 500
print_interval = 5
lr = 0.00002
b1 = 0.5
b2 = 0.999
input_dim = total_data.shape[1]
output_dim = total_label.shape[1]
d_model = 256
nhead = 4
n_layer =6
tensor_type = torch.cuda.FloatTensor

# log_write = open(f"./log/log_{pt}.txt", "w") 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = transformer.Model(
    input_dim=input_dim,
    output_dim=output_dim,
    d_model=d_model,
    nhead=nhead,
    n_layer=n_layer
).to(device)

# criterion = nn.MSELoss(reduction='mean').to(device)
criterion = nn.L1Loss(reduction='mean').to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=lr,betas=(b1,b2))

print(input_dim,output_dim)

127 23


In [101]:
train_data = torch.from_numpy(train_data)
train_label = torch.from_numpy(train_label)
test_data = torch.from_numpy(test_data).to(device).type(tensor_type)
test_label = torch.from_numpy(test_label).to(device).type(tensor_type)

train_dataset = TensorDataset(train_data,train_label)
# test_dataset = TensorDataset(test_data,test_label)

train_dataloader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True,pin_memory=True)


In [102]:
import tqdm
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(f'./logs/{pt}')

writer.add_graph(model,test_data)

In [103]:
pbar = tqdm.trange(epochs, desc=f"Epochs")

for e in pbar:
    model.train()
    aver_loss= 0
    for _, (data, label) in enumerate(train_dataloader):
        data = data.to(device)
        data = data.type(tensor_type)
        label = label.to(device)
        label = label.type(tensor_type)
        outputs = model(data)
        loss = criterion(outputs, label)
        aver_loss+=loss.detach().cpu().numpy()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # if (e + 1) % print_interval == 0:
    model.eval()
    test_outputs = model(test_data)
    test_loss = criterion(test_outputs, test_label).detach().cpu().numpy()
    aver_loss = aver_loss/len(train_dataloader)
    pbar.set_postfix({'average loss':aver_loss,'test loss':test_loss})
    writer.add_scalar('average loss',aver_loss,e)
    writer.add_scalar('test loss',test_loss,e)
    # log_write.write(f'{e}\t\t{aver_loss}\t\t{test_loss}\n')
    # log_write.flush()

torch.save(model.state_dict(), f'./res/{pt}_model.pth')
writer.close()

Epochs: 100%|██████████| 500/500 [11:09<00:00,  1.34s/it, average loss=0.41, test loss=0.42626652] 


In [104]:
test_outputs = model(test_data)
print(test_outputs.detach().cpu().numpy(),test_label.detach().cpu().numpy())

[[ 9.961931  11.13282   10.161477  ...  7.2029147  7.150252   5.8536253]
 [ 6.4487047  7.6826696  7.240362  ...  6.1469207  6.1527495  5.042369 ]
 [ 9.669281  10.917458  10.671871  ...  4.135633   3.9088254  2.5275223]
 ...
 [ 9.577301  10.978248   9.712202  ...  5.289433   5.005707   3.307084 ]
 [10.475774  11.653199  11.601434  ...  7.5067387  6.7015386  4.6305666]
 [10.5554905 11.91072   11.475509  ...  7.569194   7.035995   5.274171 ]] [[ 9.479337  11.043268   9.720101  ...  6.7097225  7.0673895  6.117388 ]
 [ 6.017125   7.5928135  7.3243685 ...  5.7848377  6.012065   4.6821566]
 [ 9.51248   10.848118  11.4815855 ...  4.1983466  3.8367896  2.5746264]
 ...
 [10.177409  11.322115   9.745621  ...  5.5244346  4.768065   3.2150702]
 [11.107904  12.544627  11.612778  ...  6.9679823  6.0626163  3.8187833]
 [10.546551  11.719819  12.143424  ...  7.2116675  6.4691424  4.6362104]]
